In [146]:
!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org tabulate
!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org seaborn
%matplotlib inline
import numpy as np
import pandas as pd
import pylab as pl
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from tabulate import tabulate
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics

You are using pip version 19.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 19.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [147]:
# Función para visualizar un conjunto de datos en 2D
def plot_data(X, y, size=None):
    y_unique = np.unique(y)
    colors = pl.cm.rainbow(np.linspace(0.0, 1.0, y_unique.size))
    for this_y, color in zip(y_unique, colors):
        this_X = X[y == this_y]
        pl.scatter(this_X[:, 0], this_X[:, 1],  c=color,
                   alpha=0.5, edgecolor='k', s=size,
                    label="Class %s" % this_y)
    pl.legend(loc="best")
    pl.title("Data")
    
# Función para visualizar de la superficie de decisión de un clasificador
def plot_decision_region2(X, pred_fun):      #Función para visualizar la superficie de decisión de nuestro algoritmo.
    min_x = np.min(X[:, 0])
    max_x = np.max(X[:, 0])
    min_y = np.min(X[:, 1])
    max_y = np.max(X[:, 1])
    min_x = min_x - (max_x - min_x) * 0.05
    max_x = max_x + (max_x - min_x) * 0.05
    min_y = min_y - (max_y - min_y) * 0.05
    max_y = max_y + (max_y - min_y) * 0.05
    x_vals = np.linspace(min_x, max_x, 100)
    y_vals = np.linspace(min_y, max_y, 100)
    XX, YY = np.meshgrid(x_vals, y_vals)
    grid_r, grid_c = XX.shape
    ZZ = np.zeros((grid_r, grid_c))
    for i in range(grid_r):
        for j in range(grid_c):
            ZZ[i, j] = pred_fun(XX[i, j], YY[i, j])
    pl.contourf(XX, YY, ZZ, 100, cmap = pl.cm.coolwarm, vmin= 1, vmax=3)
    pl.colorbar()
    pl.xlabel("x")
    pl.ylabel("y")
    
def gen_pred_fun(clf):
    def pred_fun(x1, x2):
        x = np.array([[x1, x2]])
        return clf.predict(x)[0]
    return pred_fun

def list_cm(cm,classes):     #función para generar de una forma más visual la matriz de confusión
    if len(cm)==2:
      cm.astype(int)
      row_0 =['','Valor','Verdadero']
      row_1 =['-',classes[0],classes[1]]
      row_2 =[classes[0],cm[0,0],cm[1,0]]
      row_3 =[classes[1],cm[0,1],cm[1,1]]
      table = zip(row_0,row_1, row_2, row_3)
      headers = ['', '', 'Valor', 'Predicho']  
      return print(tabulate(table, headers=headers, floatfmt=".0f"))
    else:
      cm.astype(int)
      row_0 =['','Valor','Verdadero','']
      row_1 =['-',np.int(classes[0]),classes[1],classes[2]]
      row_2 =[classes[0],cm[0,0],cm[1,0],cm[2,0]]
      row_3 =[classes[1],cm[0,1],cm[1,1],cm[2,1]]
      row_4 =[classes[2],cm[0,2],cm[1,2],cm[2,2]]
      table = zip(row_0,row_1, row_2, row_3, row_4)
      headers = ['', '', 'Valor', 'Predicho', '']  
      return print(tabulate(table, headers=headers, floatfmt=".0f")) 

import itertools

def plot_confusion_matrix(cm, classes,
                          title='Matriz de confusión',
                          cmap=pl.cm.Blues):
    print(cm) # Confusion matrix

    pl.imshow(cm, interpolation='nearest', cmap=cmap) # Pintamos la matriz como una imagen
    pl.title(title)
    pl.colorbar()
    tick_marks = np.arange(len(classes))
    pl.xticks(tick_marks, classes, rotation=45) # Nombre de las clases en X
    pl.yticks(tick_marks, classes) # Nombre de las clases en Y

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        pl.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black") 
        # Anotamos cada sección de la imagen con su valor correspondiente en la matriz

    pl.tight_layout()
    pl.ylabel('Valor de verdad')
    pl.xlabel('Valor predicho')

    
    %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
from sklearn.datasets import make_circles
from matplotlib.colors import Normalize


# Función para visualizar un conjunto de datos en 2D
def plot_data(X, y):
    y_unique = np.unique(y)
    colors = pl.cm.rainbow(np.linspace(0.0, 1.0, y_unique.size))
    for this_y, color in zip(y_unique, colors):
        this_X = X[y == this_y]
        pl.scatter(this_X[:, 0], this_X[:, 1],  c=color,
                    alpha=0.5, edgecolor='k',
                    label="Class %s" % this_y)
    pl.legend(loc="best")
    pl.title("Data")
    
# Función para visualizar de la superficie de decisión de un clasificador
def plot_decision_region(X, pred_fun):
    min_x = np.min(X[:, 0])
    max_x = np.max(X[:, 0])
    min_y = np.min(X[:, 1])
    max_y = np.max(X[:, 1])
    min_x = min_x - (max_x - min_x) * 0.05
    max_x = max_x + (max_x - min_x) * 0.05
    min_y = min_y - (max_y - min_y) * 0.05
    max_y = max_y + (max_y - min_y) * 0.05
    x_vals = np.linspace(min_x, max_x, 100)
    y_vals = np.linspace(min_y, max_y, 100)
    XX, YY = np.meshgrid(x_vals, y_vals)
    grid_r, grid_c = XX.shape
    ZZ = np.zeros((grid_r, grid_c))
    for i in range(grid_r):
        for j in range(grid_c):
            ZZ[i, j] = pred_fun(XX[i, j], YY[i, j])
    pl.contourf(XX, YY, ZZ, 100, cmap = pl.cm.coolwarm, vmin= -1, vmax=2)
    pl.colorbar()
    pl.xlabel("x")
    pl.ylabel("y")
    
class MidpointNormalize(Normalize):

    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))
    
def gen_pred_fun(clf):
    def pred_fun(x1, x2):
        x = np.array([[x1, x2]])
        return clf.predict(x)[0]
    return pred_fun

def list_cm(cm,classes):     #función para generar de una forma más visual la matriz de confusión
    if len(cm)==2:
      cm.astype(int)
      row_0 =['','Valor','Verdadero']
      row_1 =['-',classes[0],classes[1]]
      row_2 =[classes[0],cm[0,0],cm[1,0]]
      row_3 =[classes[1],cm[0,1],cm[1,1]]
      table = zip(row_0,row_1, row_2, row_3)
      headers = ['', '', 'Valor', 'Predicho']  
      return print(tabulate(table, headers=headers, floatfmt=".0f"))
    else:
      cm.astype(int)
      row_0 =['','Valor','Verdadero','']
      row_1 =['-',np.int(classes[0]),classes[1],classes[2]]
      row_2 =[classes[0],cm[0,0],cm[1,0],cm[2,0]]
      row_3 =[classes[1],cm[0,1],cm[1,1],cm[2,1]]
      row_4 =[classes[2],cm[0,2],cm[1,2],cm[2,2]]
      table = zip(row_0,row_1, row_2, row_3, row_4)
      headers = ['', '', 'Valor', 'Predicho', '']
      return print(tabulate(table, headers=headers, floatfmt=".0f")) 

In [148]:
import os
os.getcwd()

'C:\\Users\\juantrma\\OneDrive - Caja Colombiana de Subsidio Familiar - Colsubsidio\\Modelo Natación'

In [149]:
import pandas as pd
import numpy as np
import csv
df = pd.read_csv("BD/data.csv", encoding = 'latin1', sep = ",")

In [150]:
df.head(3)

,nodocumento,genero,clasedecontrato,nodeprorrogas,regional,departamento,ciudad,ubicacion,tipodevinculo,tipodeposicion,...,ryt_hoathan,ryt_hopaipa,ryt_hopeaalisa,ryt_piscilago,ryt_prdeport,ryt_prrecryt,salud_nopos,salud_pos,riesgo,mesesfalta
0,1.112537e-308,M,FIJO,0,CENTRO,BOGOTAÂ,BOGOTAÂ D.C.,PEND.REG. CENTRO,COLSUBSIDIO,OCASIONAL,...,0,0,0,0,0,0,0,0,X0,8.0
1,1.112537e-308,F,APRENDIZAJE,0,CENTRO,BOGOTAÂ,BOGOTAÂ D.C.,OFICINAS CENTRALES CALLE 26,COLSUBSIDIO,ESTUDIANTE EN PRACTICA,...,0,0,0,0,0,0,0,0,X0,5.0
2,1.112537e-308,F,APRENDIZAJE,0,CENTRO,BOGOTAÂ,BOGOTAÂ D.C.,PEND.REG. CENTRO,COLSUBSIDIO,ESTUDIANTE EN PRACTICA,...,0,0,0,0,0,0,0,0,X0,5.0


In [151]:
df.columns

Index(['nodocumento', 'genero', 'clasedecontrato', 'nodeprorrogas', 'regional',
       'departamento', 'ciudad', 'ubicacion', 'tipodevinculo',
       'tipodeposicion', 'motivodeaprobaciongral', 'turnoposicion',
       'horassemanalespromedio', 'familiasdecargo', 'divisiondepersonal',
       'subdivisiondepersonal', 'areaorganizativa', 'edad', 'salario',
       'categoria', 'segmento_poblacional', 'total_numero_grupo_familiar',
       'numero_beneficiario_cuota_monetaria', 'numero_cuota_monetaria',
       'numero_hijos', 'numero_padre', 'numero_conyugue', 'numero_hermano',
       'estado_civil', 'nivel_academico', 'segmento_grupo_familiar',
       'cx_persona', 'cy_persona', 'departamentopersona', 'municipiopersona',
       'poblado', 'tipopoblado', 'latitudpoblado', 'longitudpoblado',
       'estratopersona', 'filial_famisanar', 'filial_ips_colsubsidio',
       'filial_ips_cafam', 'filial_otra_ips', 'filial_pac_famisanar',
       'filial_bancompartir', 'marca_aventurero', 'marca_tu_clu

In [152]:
categoricas=['genero', 'clasedecontrato', 'regional',
       'departamento', 'ciudad', 'ubicacion', 'tipodevinculo',
       'tipodeposicion', 'motivodeaprobaciongral', 'turnoposicion',
             'familiasdecargo', 'divisiondepersonal',
       'subdivisiondepersonal', 'areaorganizativa',
       'categoria', 'segmento_poblacional','estado_civil', 'nivel_academico', 'segmento_grupo_familiar',
       'departamentopersona', 'municipiopersona',
       'poblado', 'tipopoblado', 
       'estratopersona', 'filial_famisanar', 'filial_ips_colsubsidio',
       'filial_ips_cafam', 'filial_otra_ips', 'filial_pac_famisanar',
       'filial_bancompartir', 'marca_aventurero', 'marca_tu_club',
       'filial_suramericana', 'filial_pac_suramericana', 'filial_proteccion',
       'filial_pension_obligatoria', 'filial_pension_voluntaria',
       'filial_cesantias', 'cs_as_hipo', 'cs_as_libr',
       'cs_as_nolibranza', 'cs_as_titular', 'cs_co_csconv', 'ms_drog',
       'ms_sup', 'ryt_agviaje', 'ryt_clbellavista', 'ryt_clcll195',
       'ryt_clcolina', 'ryt_clcubo', 'ryt_hoalcaravan', 'ryt_hoathan',
       'ryt_hopaipa', 'ryt_hopeaalisa', 'ryt_piscilago', 'ryt_prdeport',
       'ryt_prrecryt', 'salud_nopos', 'salud_pos', 'riesgo']

for var in categoricas:
    df[var] = df[var].astype("category")

In [153]:
df=df.drop(['nodocumento'], axis=1)

In [155]:
df.apply(lambda x: x.isnull().sum(), axis=0)

Index(['genero', 'clasedecontrato', 'nodeprorrogas', 'regional',
       'departamento', 'ciudad', 'ubicacion', 'tipodevinculo',
       'tipodeposicion', 'motivodeaprobaciongral', 'turnoposicion',
       'horassemanalespromedio', 'familiasdecargo', 'divisiondepersonal',
       'subdivisiondepersonal', 'areaorganizativa', 'edad', 'salario',
       'categoria', 'segmento_poblacional', 'total_numero_grupo_familiar',
       'numero_beneficiario_cuota_monetaria', 'numero_cuota_monetaria',
       'numero_hijos', 'numero_padre', 'numero_conyugue', 'numero_hermano',
       'estado_civil', 'nivel_academico', 'segmento_grupo_familiar',
       'cx_persona', 'cy_persona', 'departamentopersona', 'municipiopersona',
       'poblado', 'tipopoblado', 'latitudpoblado', 'longitudpoblado',
       'estratopersona', 'filial_famisanar', 'filial_ips_colsubsidio',
       'filial_ips_cafam', 'filial_otra_ips', 'filial_pac_famisanar',
       'filial_bancompartir', 'marca_aventurero', 'marca_tu_club',
       'fil

In [168]:
categoricas=['genero', 'clasedecontrato', 'regional',
       'departamento', 'ciudad', 'ubicacion', 'tipodevinculo',
       'tipodeposicion', 'motivodeaprobaciongral', 'turnoposicion',
             'familiasdecargo', 'divisiondepersonal',
       'subdivisiondepersonal', 'areaorganizativa',
       'categoria', 'segmento_poblacional','estado_civil', 'nivel_academico', 'segmento_grupo_familiar',
       'departamentopersona', 'municipiopersona',
       'poblado', 'tipopoblado', 
       'estratopersona', 'filial_famisanar', 'filial_ips_colsubsidio',
       'filial_ips_cafam', 'filial_otra_ips', 'filial_pac_famisanar',
       'filial_bancompartir', 'marca_aventurero', 'marca_tu_club',
       'filial_suramericana', 'filial_pac_suramericana', 'filial_proteccion',
       'filial_pension_obligatoria', 'filial_pension_voluntaria',
       'filial_cesantias', 'cs_as_hipo', 'cs_as_libr',
       'cs_as_nolibranza', 'cs_as_titular', 'cs_co_csconv', 'ms_drog',
       'ms_sup', 'ryt_agviaje', 'ryt_clbellavista', 'ryt_clcll195',
       'ryt_clcolina', 'ryt_clcubo', 'ryt_hoalcaravan', 'ryt_hoathan',
       'ryt_hopaipa', 'ryt_hopeaalisa', 'ryt_piscilago', 'ryt_prdeport',
       'ryt_prrecryt', 'salud_nopos', 'salud_pos']

In [169]:
df1 = pd.get_dummies(df, columns=categoricas, prefix=['genero', 'clasedecontrato', 'regional',
       'departamento', 'ciudad', 'ubicacion', 'tipodevinculo',
       'tipodeposicion', 'motivodeaprobaciongral', 'turnoposicion',
             'familiasdecargo', 'divisiondepersonal',
       'subdivisiondepersonal', 'areaorganizativa',
       'categoria', 'segmento_poblacional','estado_civil', 'nivel_academico', 'segmento_grupo_familiar',
       'departamentopersona', 'municipiopersona',
       'poblado', 'tipopoblado', 
       'estratopersona', 'filial_famisanar', 'filial_ips_colsubsidio',
       'filial_ips_cafam', 'filial_otra_ips', 'filial_pac_famisanar',
       'filial_bancompartir', 'marca_aventurero', 'marca_tu_club',
       'filial_suramericana', 'filial_pac_suramericana', 'filial_proteccion',
       'filial_pension_obligatoria', 'filial_pension_voluntaria',
       'filial_cesantias', 'cs_as_hipo', 'cs_as_libr',
       'cs_as_nolibranza', 'cs_as_titular', 'cs_co_csconv', 'ms_drog',
       'ms_sup', 'ryt_agviaje', 'ryt_clbellavista', 'ryt_clcll195',
       'ryt_clcolina', 'ryt_clcubo', 'ryt_hoalcaravan', 'ryt_hoathan',
       'ryt_hopaipa', 'ryt_hopeaalisa', 'ryt_piscilago', 'ryt_prdeport',
       'ryt_prrecryt', 'salud_nopos', 'salud_pos'])
#df1 = df
X = df1.drop(["riesgo"], axis = 1)
y = df1[["riesgo"]]

## Train & Test

In [170]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=31415)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(12217, 1136) (5236, 1136) (12217, 1) (5236, 1)


### Estimación de Modelos

In [171]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import LinearSVC 
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [172]:
models = {
    'GradientBoosting': GradientBoostingClassifier(random_state=31415, n_estimators=1250)
    ,'Classifier_ovr': LogisticRegression(solver='lbfgs', multi_class = 'ovr')
    ,'RandomForest' : RandomForestClassifier()
    ,'KNN' : KNeighborsClassifier(n_neighbors=65)
    ,'DesicionTree' : DecisionTreeClassifier(max_depth=5)
}

for model in models.keys():
    models[model].fit(X_train, y_train)

c:\users\juantrma\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\juantrma\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\juantrma\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\juantrma\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: DataConversionWarning: A column

In [173]:
y_pred = pd.DataFrame(index=X_test.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)

In [174]:
classes = df['riesgo'].unique()
print(classes)
for model in models.keys():
    cnf_matrix_1 = confusion_matrix(y_test, y_pred[model])
    print(model)
    list_cm(cnf_matrix_1, classes = classes)

[X0, X1]
Categories (2, object): [X0, X1]
GradientBoosting
               Valor    Predicho
---------  --  -------  ----------
           -   X0       X1
Valor      X0  5121     32
Verdadero  X1  69       14
Classifier_ovr
               Valor    Predicho
---------  --  -------  ----------
           -   X0       X1
Valor      X0  5153     0
Verdadero  X1  83       0
RandomForest
               Valor    Predicho
---------  --  -------  ----------
           -   X0       X1
Valor      X0  5147     6
Verdadero  X1  81       2
KNN
               Valor    Predicho
---------  --  -------  ----------
           -   X0       X1
Valor      X0  5153     0
Verdadero  X1  83       0
DesicionTree
               Valor    Predicho
---------  --  -------  ----------
           -   X0       X1
Valor      X0  5147     6
Verdadero  X1  77       6


In [175]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

for model in models.keys():
    print(model)
    print('Accurracy: {}'.format(models[model].score(X_test, y_test)))
    print('Error de clasificación: {}'.format(1 - models[model].score(X_test, y_test)))
    print('Precisión macro: {}'.format(precision_score(y_test, y_pred[model], average='macro')))
    print('Recall macro: {}'.format(recall_score(y_test, y_pred[model], average='macro')))
    print('F1 macro: {}'.format(f1_score(y_test,y_pred[model], average='macro')))
    print('AUC: {}'.format(roc_auc_score(y_test, models[model].predict_proba(X_test)[:, 1]), average='macro'))

GradientBoosting
Accurracy: 0.9807104660045837
Error de clasificación: 0.01928953399541633
Precisión macro: 0.6455265141995477
Recall macro: 0.5812323620116016
F1 macro: 0.6036446025361122
AUC: 0.8974091592451701
Classifier_ovr
Accurracy: 0.98414820473644
Error de clasificación: 0.015851795263559953
Precisión macro: 0.49207410236822
Recall macro: 0.5
F1 macro: 0.4960053903166811


c:\users\juantrma\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\juantrma\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


AUC: 0.7100601123687452
RandomForest
Accurracy: 0.9833842627960275
Error de clasificación: 0.01661573720397247
Precisión macro: 0.6172532517214997
Recall macro: 0.5114660076362114
F1 macro: 0.5177876742273236
AUC: 0.7673995029214472
KNN
Accurracy: 0.98414820473644
Error de clasificación: 0.015851795263559953
Precisión macro: 0.49207410236822
Recall macro: 0.5
F1 macro: 0.4960053903166811


c:\users\juantrma\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\juantrma\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


AUC: 0.7893366596601816
DesicionTree
Accurracy: 0.98414820473644
Error de clasificación: 0.015851795263559953
Precisión macro: 0.7426301684532925
Recall macro: 0.5355623931783801
F1 macro: 0.5591586656725653
AUC: 0.8128473529281107


## Cross Validation

In [177]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

FinalMod=models['GradientBoosting']

n_cv=10
scoring = ['accuracy', 'f1_macro', 'precision_macro', 'recall_macro', 'roc_auc']

score1=cross_validate(FinalMod, X_train, y_train, cv=n_cv, scoring=scoring)
print("train MSE: \n", score1, "\n")

score1=cross_validate(FinalMod, X_test, y_test, cv=n_cv, scoring=scoring)
print("test MSE: \n", score1)

c:\users\juantrma\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\juantrma\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\juantrma\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\juantrma\appdata\local\programs\python\python37\lib\site-packages\s

train MSE Bellavista: 
 {'fit_time': array([209.60901642, 208.04771924, 206.55452609, 204.86565495,
       211.01104283, 214.6269691 , 209.99385762, 203.77270031,
       207.3506968 , 223.25328255]), 'score_time': array([0.19550562, 0.18542695, 0.18546987, 0.19647479, 0.20546675,
       0.20445275, 0.23437285, 0.19051909, 0.19698071, 0.23036194]), 'test_accuracy': array([0.9787408 , 0.97790507, 0.98199673, 0.97790507, 0.97381342,
       0.98036007, 0.97870598, 0.98034398, 0.97706798, 0.97788698]), 'test_f1_macro': array([0.58836475, 0.55892143, 0.65168947, 0.55892143, 0.54891104,
       0.66168328, 0.53032903, 0.62002075, 0.55669087, 0.55891679]), 'test_precision_macro': array([0.61798101, 0.58347722, 0.70213499, 0.58347722, 0.55503731,
       0.68169569, 0.54812981, 0.64763754, 0.56988665, 0.57630273]), 'test_recall_macro': array([0.57125935, 0.54625624, 0.62208819, 0.54625624, 0.54417637,
       0.64584027, 0.522988  , 0.6015194 , 0.54805587, 0.54847185]), 'test_roc_auc': array([0.87

c:\users\juantrma\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\juantrma\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\juantrma\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\juantrma\appdata\local\programs\python\python37\lib\site-packages\s

test MSE Bellavista: 
 {'fit_time': array([74.74500632, 74.60737467, 72.46809745, 72.9557941 , 73.15825152,
       72.27262068, 73.39362168, 72.76231146, 73.7885654 , 72.44316459]), 'score_time': array([0.0847733 , 0.0847733 , 0.08277893, 0.08377647, 0.09873652,
       0.08178139, 0.08178186, 0.07779193, 0.08277869, 0.07878923]), 'test_accuracy': array([0.97333333, 0.98285714, 0.9752381 , 0.98852772, 0.98279159,
       0.98279159, 0.98279159, 0.98087954, 0.97705545, 0.9789675 ]), 'test_f1_macro': array([0.55573017, 0.586578  , 0.61135471, 0.74709865, 0.49566056,
       0.64948991, 0.64948991, 0.49517375, 0.66083009, 0.70517091]), 'test_precision_macro': array([0.56370656, 0.74235182, 0.61823017, 0.87018304, 0.49233716,
       0.69420849, 0.69420849, 0.49232246, 0.64512671, 0.67791193]), 'test_recall_macro': array([0.5497416 , 0.55458656, 0.60529716, 0.68652913, 0.49902913,
       0.62208738, 0.62208738, 0.49805825, 0.68070388, 0.74320388]), 'test_roc_auc': array([0.93690784, 0.83850129

In [ ]:
Proba = FinalMod.predict_proba(X_test)